In [4]:
# result_andicator=result_andicator[result_andicator['sharp_ratio']!=np.inf]
# result_andicator=result_andicator.sort_values('decision',ascending=False)
# result_andicator[result_andicator['number_profit_trade']>result_andicator['number_lose_trade']]
# result_andicator.iloc[0:60]

import pandas as pd
import numpy as np
import os
import sys #, traceback, logging
import threading
# address='F:/ارز دیجیتال-فارکس/Python Scripts/ccxt/module/bot/آنالیز دیتا'
# raw_data=pd.DataFrame(columns=['raw_data'],index=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
# result=pd.DataFrame(columns=['result','decision'],index=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,'sum_decision'])

def calcu_raw_data(alldata,q):
    print(q,end='-')
    global raw_data

    values=[]
    column=[]
    number_profit_trade=[]
    number_lose_trade=[]
    numberwin={}
    s_l_st=[]
    meanvalues=[]
    devition=[]
    leverage_list=[]
    risk_list=[]
    risk_free=0
    for n in alldata.columns:
        number_positive_value=[]
        for j in alldata.loc['result_value',n].columns:
            for i in alldata.loc['result_value',n].index:
                if type(alldata.loc['result_order',n].loc[i,j])==float:
                    continue
                values.append(alldata.loc['result_value',n].loc[i,j])
                number_profit_trade.append(len(alldata.loc['result_profit',n].loc[i,j]))
                number_lose_trade.append(len(alldata.loc['result_lose',n].loc[i,j]))
                if alldata.loc['result_value',n].loc[i,j] > 100:
                    number_positive_value.append(0)
                column.append(i+'*'+j)
                s_l_st.append(n)

                meanvalues.append((alldata.loc['result_lose',n].loc[i,j]['lose'].sum() + alldata.loc['result_profit',n].loc[i,j]['gain'].sum())\
                /(len(alldata.loc['result_lose',n].loc[i,j]) + len(alldata.loc['result_profit',n].loc[i,j])))
                tp=list(alldata.loc['result_lose',n].loc[i,j]['lose'])
                tp.extend(alldata.loc['result_profit',n].loc[i,j]['gain'])
                devition.append(np.std(tp))

                leverage_list.append(int(n.split("-")[1]))
                risk_list.append(float(n.split("-")[0]))
        numberwin[n]=len(number_positive_value)

    result_andicator=pd.DataFrame(index=['s_l_st','values','values(%)','number_profit_trade','number_lose_trade','meanvalues','devition',
    'sharp_ratio','numberwin','dif_Np_Nl','risk_list','leverage_list','decision','all_trade','control'],columns=column)

    result_andicator.loc['s_l_st',:] = s_l_st
    result_andicator.loc['values',:] = values
    result_andicator.loc['number_profit_trade',:] = number_profit_trade
    result_andicator.loc['number_lose_trade',:] = number_lose_trade
    result_andicator.loc['meanvalues',:] = meanvalues
    result_andicator.loc['devition',:] = devition
    result_andicator.loc['risk_list',:] = risk_list
    result_andicator.loc['leverage_list',:] = leverage_list

    result_andicator=result_andicator.T.copy()
    number_col=list(result_andicator.columns)

    result_andicator['values(%)']=(result_andicator['values']-100)/100
    result_andicator['values']=result_andicator['values']-100
    result_andicator['all_trade']=result_andicator['number_profit_trade'] + result_andicator['number_lose_trade']
    sample=result_andicator
    # result_andicator.set_index('s_l_st',append=True, inplace=True)
    for i in list(numberwin.keys()):
        l=0
        for j in result_andicator['s_l_st']:
            if j==i:
                result_andicator.iloc[l,number_col.index('numberwin')] = numberwin[i]
            l+=1

    result_andicator['sharp_ratio']=(result_andicator['meanvalues'] - risk_free)/result_andicator['devition']

    result_andicator['dif_Np_Nl']=result_andicator['number_profit_trade'] - result_andicator['number_lose_trade']

    raw_data.at[q,'raw_data']=result_andicator

    return

def run_calcu_raw_data():
    global address
    global raw_data
    threads=[]
    for q in raw_data.index:
        try:

            alldata = pd.read_pickle('%s/alldata%s.pkl'% (address,q+1))

        except Exception as error:
            break  
        thread=threading.Thread(target=calcu_raw_data,args=(alldata,q,))

        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()
    complet_raw_data()
    return

def complet_raw_data():
    global raw_data
    # setting index
    for i in raw_data.index:
        try:
            raw_data.loc[i,'raw_data'].index.name = 'andicaor'
            raw_data.loc[i,'raw_data'].set_index('s_l_st',append=True, inplace=True)
        except Exception as error:
            break

    # remove trash values
    for i in raw_data.index:
        print(i)
        try:
            raw_data.loc[i,'raw_data']['sharp_ratio'].replace([np.inf, -np.inf], 0, inplace=True)
            raw_data.loc[i,'raw_data']['sharp_ratio'].fillna(0, inplace=True)
        except Exception as error:
            print(error)
            break

    raw_data.to_pickle('%s/raw_data.pkl'% (address))
    
def calcu_result(result_andicator,q,weight):
    global result
    print(q,end='-')
    number_col=list(result_andicator.columns)
    weight=weight
    nt=3
    pnt=10
    sr=3
    nv=0.1
    nsl=0.01

    for i in range(len(result_andicator)):

        if (result_andicator.iloc[i,number_col.index('number_profit_trade')] + result_andicator.iloc[i,number_col.index('number_lose_trade')])<=nt\
        or (result_andicator.iloc[i,number_col.index('values(%)')]<=nv and result_andicator.iloc[i,number_col.index('values(%)')]>0):

            if (result_andicator.iloc[i,number_col.index('number_profit_trade')] + result_andicator.iloc[i,number_col.index('number_lose_trade')])<=nt\
        and (result_andicator.iloc[i,number_col.index('values(%)')]<=nv and result_andicator.iloc[i,number_col.index('values(%)')]>0):
                result_andicator.iloc[i,number_col.index('control')]='nv-nt'

                result_andicator.iloc[i,number_col.index('decision')]=(nsl/result_andicator.iloc[i,number_col.index('risk_list')])*weight['wsl']\
                + (1/result_andicator.iloc[i,number_col.index('leverage_list')])*weight['wL'] \
                + (result_andicator.iloc[i,number_col.index('values(%)')]/nv)*weight['wv'] \
                + (result_andicator.iloc[i,number_col.index('dif_Np_Nl')]/(result_andicator.iloc[i,number_col.index('all_trade')] + pnt))*weight['wp'] \
                + (result_andicator.iloc[i,number_col.index('sharp_ratio')]/sr)*weight['wsr']
            else:
                if (result_andicator.iloc[i,number_col.index('number_profit_trade')] + result_andicator.iloc[i,number_col.index('number_lose_trade')])<=nt:
                    result_andicator.iloc[i,number_col.index('control')]='nt'

                    result_andicator.iloc[i,number_col.index('decision')]=(nsl/result_andicator.iloc[i,number_col.index('risk_list')])*weight['wsl']\
                    + (1/result_andicator.iloc[i,number_col.index('leverage_list')])*weight['wL'] \
                    + (result_andicator.iloc[i,number_col.index('values(%)')]/1)*weight['wv'] \
                    + (result_andicator.iloc[i,number_col.index('dif_Np_Nl')]/(result_andicator.iloc[i,number_col.index('all_trade')] + pnt))*weight['wp'] \
                    + (result_andicator.iloc[i,number_col.index('sharp_ratio')]/sr)*weight['wsr']


                elif (result_andicator.iloc[i,number_col.index('values(%)')]<=nv and result_andicator.iloc[i,number_col.index('values(%)')]>0):
                    result_andicator.iloc[i,number_col.index('control')]='nv'

                    result_andicator.iloc[i,number_col.index('decision')]=(nsl/result_andicator.iloc[i,number_col.index('risk_list')])*weight['wsl']\
                    + (1/result_andicator.iloc[i,number_col.index('leverage_list')])*weight['wL'] \
                    + (result_andicator.iloc[i,number_col.index('values(%)')]/nv)*weight['wv'] \
                    + (result_andicator.iloc[i,number_col.index('dif_Np_Nl')]/(result_andicator.iloc[i,number_col.index('all_trade')]))*weight['wp'] \
                    + (result_andicator.iloc[i,number_col.index('sharp_ratio')]/sr)*weight['wsr']                

        else:
            result_andicator.iloc[i,number_col.index('control')]='-'

            result_andicator.iloc[i,number_col.index('decision')]=(nsl/result_andicator.iloc[i,number_col.index('risk_list')])*weight['wsl']\
            + (1/result_andicator.iloc[i,number_col.index('leverage_list')])*weight['wL'] \
            + (result_andicator.iloc[i,number_col.index('values(%)')]/1)*weight['wv'] \
            + (result_andicator.iloc[i,number_col.index('dif_Np_Nl')]/(result_andicator.iloc[i,number_col.index('all_trade')]))*weight['wp'] \
            + (result_andicator.iloc[i,number_col.index('sharp_ratio')]/sr)*weight['wsr']
            
    result.at[q,'result']=result_andicator        

    return 
    
def run_calcu_result(weight):
    global address
    raw_data = pd.read_pickle('%s/raw_data.pkl'% (address))
    threads=[]
    for q in raw_data.index:
        try:

            result_andicator=pd.DataFrame(raw_data.loc[q,'raw_data'].copy())
        except Exception as error:
            print(error)
            break
        thread=threading.Thread(target=calcu_result,args=(result_andicator,q,weight,))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()
    complet_result()

def complet_result():
    global result
    for i in result.index:
        print(i)
        try:
    #         result.loc[i,'result']['decision'].mask(result.loc[i,'result']['values'] <0, 0, inplace=True)
            result.iat[i,1] = pd.DataFrame(result.loc[i,'result']['decision'].copy())
        except Exception as error:
            print(error)
            break

    sum_decision=pd.DataFrame(result.loc[0,'decision'].copy())
    for i in result.index:
        print(i)
        try:
            sum_decision=pd.DataFrame(sum_decision['decision']+pd.DataFrame(result.loc[i,'decision'])['decision'])
        except Exception as error:
            print(error)
            break
    result.iat[-1,1]=sum_decision
    result.to_pickle('%s/resultpkl'% (address))
    return

def get_best_strategy_decision():
    global result
    value_profit_strategy={}
    dif_Np_Nl={}
    all_trade={}
    dec=result.loc['sum_decision','decision'].copy()
    dec=dec.sort_values('decision',ascending=False)
    ind=dec.index
    for i in ind:
        va=[]
        Np_Nl=[]
        trade=[]
        for j in result.index:
            try:
                if j==0:
                    continue
                va.append(result.loc[j,'result'].loc[i,'values'])
                Np_Nl.append(result.loc[j,'result'].loc[i,'dif_Np_Nl'])
                trade.append(result.loc[j,'result'].loc[i,'all_trade'])
            except Exception as error:
#                 print(error)
                break
        value_profit_strategy[i[0]+'*'+i[1]]=va
        dif_Np_Nl[i[0]+'*'+i[1]]=Np_Nl
        all_trade[i[0]+'*'+i[1]]=trade
    return value_profit_strategy , dif_Np_Nl,all_trade

def filter_strategy(value_profit_strategy , dif_Np_Nl , all_trade):
    global address
    maxi=value_profit_strategy.copy()
    sum_Np_Nl=dif_Np_Nl.copy()
    sum_trade=all_trade.copy()
    for i in value_profit_strategy.keys():
        tem=[]
        Np_Nl=0
        tr=0
        tem.append(sum(x > 0 for x in value_profit_strategy[i]))
        tem.append(sum(value_profit_strategy[i]))
        Np_Nl=sum(sum_Np_Nl[i])
        tr=sum(sum_trade[i])
        maxi[i]=tem
        sum_Np_Nl[i]=Np_Nl
        sum_trade[i]=tr
    df=pd.DataFrame(maxi).T
    df.rename({0: 'number_positive', 1: 'sum_value'}, axis=1, inplace=True)

    df['all_trade']=np.nan
    df['devition']=np.nan
    df['dif_Np_Nl']=np.nan
    df['avg']=np.nan
    df['list_positive_value']=0
    df['list_positive_value'] = df['list_positive_value'].astype(object)

    for i in value_profit_strategy.keys():
        tem=value_profit_strategy[i].copy()
        tem=[x for x in tem if x >= 0 ]
        dev=np.array(tem).std()
        df.at[i,'list_positive_value']=value_profit_strategy[i].copy()#tem
        if len(tem)!=0:
            df.at[i,'avg']=sum(tem)/len(tem)
            df.at[i,'devition']=dev
        df.at[i,'dif_Np_Nl']=sum_Np_Nl[i]
        df.at[i,'all_trade']=sum_trade[i]
    df.sort_values(['number_positive', 'sum_value','devition'], ascending=[False, False, True], inplace=True) # low risk
#     df.sort_values(['sum_value','number_positive', ], ascending=[False, False], inplace=True) # above risk
    df=df[(df['sum_value']>0)] # & (df['dif_Np_Nl']>0) #  delet negetive value
    df.to_pickle('%s/filltred_strategy.pkl'% (address))
    return df

def save_to_andicator():
    wma_signal0=result_andicator.loc['wma_signal0',:]
    wma_signal1=result_andicator.loc['wma_signal1',:]
    wma_signal2=result_andicator.loc['wma_signal2',:]
    wma_signal3=result_andicator.loc['wma_signal3',:]
    macd_signal0=result_andicator.loc['macd_signal0',:]
    macd_signal1=result_andicator.loc['macd_signal1',:]
    macd_signal2=result_andicator.loc['macd_signal2',:]
    rsi_signal0=result_andicator.loc['rsi_signal0',:]
    bbands_signal0=result_andicator.loc['bbands_signal0',:]
    obv_signal0=result_andicator.loc['obv_signal0',:]
    listdata=[wma_signal0,wma_signal1,wma_signal2,wma_signal3,macd_signal0,macd_signal1,macd_signal2,rsi_signal0
    ,bbands_signal0,obv_signal0]

    lendata={'wma_signal0':len(listdata[0]),'wma_signal1':len(listdata[1]),
             'wma_signal2':len(listdata[2]),'wma_signal3':len(listdata[3]),
             'macd_signal0':len(listdata[4]),'macd_signal1':len(listdata[5]),
             'macd_signal2':len(listdata[6]),'rsi_signal0':len(listdata[7]),
             'bbands_signal0':len(listdata[8]),'obv_signal0':len(listdata[9]),}
    print(lendata)
# # weight={'wsl':0.08,'wL':0.08,'wv':0.1,'wp':0.74,'wsr':0.000001}
# weight={'wsl':0.08,'wL':0.08,'wv':0.1,'wp':0.74,'wsr':0.0}
# run_calcu_raw_data()
# run_calcu_result(weight)
value_profit_strategy , dif_Np_Nl , all_trade = get_best_strategy_decision()
df=filter_strategy(value_profit_strategy , dif_Np_Nl , all_trade)

C:\Users\hossain\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\hossain\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\hossain\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [3]:
pd.set_option('display.max_rows', df.shape[0]+1)
pd.set_option('display.max_colwidth', None)
df#.iloc[0:60]
# df.loc['bbands*signal0*0.05-8-0.19','list_positive_value']

,number_positive,sum_value,all_trade,devition,dif_Np_Nl,avg,list_positive_value
wma*signal3*0.04-2-0.23,6.0,5.934962,8.0,1.421649,4.0,2.377170,"[3.3928741092636585, 0.21829293817236817, -4.031536388140154, 3.981165997556303, 0.8854043307256632, 3.6357836570663267, 2.149498647805302, -4.296521081489885]"
wma*signal3*0.04-2-0.06,6.0,5.934962,8.0,1.421649,4.0,2.377170,"[3.3928741092636585, 0.21829293817236817, -4.031536388140154, 3.981165997556303, 0.8854043307256632, 3.6357836570663267, 2.149498647805302, -4.296521081489885]"
wma*signal3*0.04-2-0.05,6.0,5.934962,8.0,1.421649,4.0,2.377170,"[3.3928741092636585, 0.21829293817236817, -4.031536388140154, 3.981165997556303, 0.8854043307256632, 3.6357836570663267, 2.149498647805302, -4.296521081489885]"
wma*signal3*0.04-2-0.15,6.0,5.934962,8.0,1.421649,4.0,2.377170,"[3.3928741092636585, 0.21829293817236817, -4.031536388140154, 3.981165997556303, 0.8854043307256632, 3.6357836570663267, 2.149498647805302, -4.296521081489885]"
wma*signal3*0.04-2-0.24,6.0,5.934962,8.0,1.421649,4.0,2.377170,"[3.3928741092636585, 0.21829293817236817, -4.031536388140154, 3.981165997556303, 0.8854043307256632, 3.6357836570663267, 2.149498647805302, -4.296521081489885]"
wma*signal3*0.04-2-0.12,6.0,5.934962,8.0,1.421649,4.0,2.377170,"[3.3928741092636585, 0.21829293817236817, -4.031536388140154, 3.981165997556303, 0.8854043307256632, 3.6357836570663267, 2.149498647805302, -4.296521081489885]"
wma*signal3*0.04-2-0.17,6.0,5.934962,8.0,1.421649,4.0,2.377170,"[3.3928741092636585, 0.21829293817236817, -4.031536388140154, 3.981165997556303, 0.8854043307256632, 3.6357836570663267, 2.149498647805302, -4.296521081489885]"
wma*signal3*0.04-2-0.13,6.0,5.934962,8.0,1.421649,4.0,2.377170,"[3.3928741092636585, 0.21829293817236817, -4.031536388140154, 3.981165997556303, 0.8854043307256632, 3.6357836570663267, 2.149498647805302, -4.296521081489885]"
wma*signal3*0.04-2-0.2,6.0,5.934962,8.0,1.421649,4.0,2.377170,"[3.3928741092636585, 0.21829293817236817, -4.031536388140154, 3.981165997556303, 0.8854043307256632, 3.6357836570663267, 2.149498647805302, -4.296521081489885]"
wma*signal3*0.04-2-0.14,6.0,5.934962,8.0,1.421649,4.0,2.377170,"[3.3928741092636585, 0.21829293817236817, -4.031536388140154, 3.981165997556303, 0.8854043307256632, 3.6357836570663267, 2.149498647805302, -4.296521081489885]"


In [ ]:
import pandas as pd
f=pd.read_pickle('sellorbuy.pkl')
f.loc['wma','signal3'].iloc[500:540]

In [ ]:
pd.read_pickle('detail_accuonts.pkl')

In [22]:
# alldata = pd.read_pickle('%s/alldata4.pkl'% (address))
andicator='wma'
signal=3
slt='0.04-2-0.01'
display(alldata.loc['result_order',slt].loc[andicator,'signal%s'%(signal)])
display(alldata.loc['result_profit',slt].loc[andicator,'signal%s'%(signal)])
display(alldata.loc['result_lose',slt].loc[andicator,'signal%s'%(signal)])
display(alldata.loc['result_value',slt].loc[andicator,'signal%s'%(signal)])
# alldata.loc['result_order','0.05-1-0.08']

,index,type position
1,1079,buy
2,1604,buy


,index,traillist,size,start-price,stopprice,open,high,low,percent profit (%),gain,type position


,index,traillist,size,start-price,stopprice,open,high,low,percent lose (%),lose,type position
1,start:1079 - end:1572,"[23930.0, 24200.0]",0.008423,23744.0,23621.0,24053.0,24053.0,23621.0,-0.52,-1.43,buy
2,start:1604 - end:1767,[24000.0],0.008363,23743.0,23566.0,23609.0,23609.0,23566.0,-0.75,-1.87,buy


96.69157370951893

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>" ))

In [ ]:
from stem import Signal
from stem.control import Controller

# signal TOR for a new connection 
def renew_connection():
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate(password="password")
        controller.signal(Signal.NEWNYM)
renew_connection()

In [ ]:
from stem import Signal
from stem.control import Controller
import requests
proxies = {'http':  'socks5://127.0.0.1:9050',
                       'https': 'socks5://127.0.0.1:9050'}
def change_ip():
    with Controller.from_port(port = 9051) as controller:
        print("IP Changed!")
        controller.authenticate()
        print("IP Changed!")
        controller.signal(Signal.NEWNYM)
        print("IP Changed!")
for i in range(5):
    res = requests.get("https://api.ipify.org", proxies=proxies)
    print(res.text)
    change_ip()
    

In [ ]:
import io
import os
import stem.process
import re
SOCKS_PORT = 9050
TOR_PATH = "C:\\Users\\hossain\\AppData\\Roaming\\tor\\tor.exe"
tor_process = stem.process.launch_tor_with_config(
  config = {
    'SocksPort': str(SOCKS_PORT),
  },
  init_msg_handler = lambda line: print(line) if re.search('Bootstrapped', line) else False,
  tor_cmd = TOR_PATH
)

In [ ]:
!pip install -U requests[socks]

# دامپ و پامپ ها را باید جدا کنم و ببینم کدوم استراتژی توانسته بهتر اونها را رصد کنه. داده های با پراکندگی زیاد را باید حذف کنم چون که اونها در دامپ و پامپ ها به وجود آمدن
